In [13]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense, GRU


# Load data from JSON file
with open("data.json") as f:
    data = json.load(f)

# Extract features and target
X = np.column_stack((data["o"], data["h"], data["l"], data["c"], data["v"]))
y = np.array(data["c"])

# Normalize data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Reshape data for LSTM input (samples, timesteps, features)
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.3, random_state=42
)

# Build the model
model = Sequential()
model.add(
    Bidirectional(
        GRU(
            units=128,
            return_sequences=True,
            input_shape=(X_scaled.shape[1], X_scaled.shape[2]),
        )
    )
)
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(units=128)))
model.add(Dropout(0.2))
model.add(Dense(32, kernel_initializer="uniform", activation="relu"))
model.add(Dense(1, kernel_initializer="uniform", activation="linear"))
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model using train data
model.fit(X_train, y_train, epochs=500, batch_size=4048, verbose=1)

# Evaluate the model on test data
loss = model.evaluate(X_test, y_test, verbose=1)
print("Test Loss:", loss)

# Predictions on test data
predictions_scaled = model.predict(X_test)
predictions = scaler_y.inverse_transform(predictions_scaled.flatten().reshape(-1, 1))
y_test_inverse = scaler_y.inverse_transform(y_test.flatten().reshape(-1, 1))

print("Predictions on Test Data:", predictions)
print("Actual values on Test Data:", y_test_inverse)

Epoch 1/500
1/1 [==============================] - 5s 5s/step - loss: 0.2056
Epoch 2/500
1/1 [==============================] - 0s 121ms/step - loss: 0.1966
Epoch 3/500
1/1 [==============================] - 0s 115ms/step - loss: 0.1880
Epoch 4/500
1/1 [==============================] - 0s 138ms/step - loss: 0.1786
Epoch 5/500
1/1 [==============================] - 0s 199ms/step - loss: 0.1678
Epoch 6/500
1/1 [==============================] - 0s 188ms/step - loss: 0.1549
Epoch 7/500
1/1 [==============================] - 0s 159ms/step - loss: 0.1392
Epoch 8/500
1/1 [==============================] - 0s 170ms/step - loss: 0.1201
Epoch 9/500
1/1 [==============================] - 0s 256ms/step - loss: 0.0975
Epoch 10/500
1/1 [==============================] - 0s 183ms/step - loss: 0.0716
Epoch 11/500
1/1 [==============================] - 0s 193ms/step - loss: 0.0440
Epoch 12/500
1/1 [==============================] - 0s 203ms/step - loss: 0.0195
Epoch 13/500
1/1 [======================

In [19]:
# Predict tomorrow's value
last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)
predicted_value_scaled = model.predict(last_data_point)
predicted_value = scaler_y.inverse_transform(predicted_value_scaled)
print("Predicted Tomorrow's Close Price:", predicted_value)

1/1 [==============================] - 0s 25ms/step
Predicted Tomorrow's Close Price: [[1281.3223]]


In [21]:
# Predict tomorrow's value
last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)
predicted_values = []

for _ in range(7):
    # Predict next day's value
    predicted_value_scaled = model.predict(last_data_point)
    predicted_value = scaler_y.inverse_transform(predicted_value_scaled)
    predicted_values.append(predicted_value.flatten()[0])

    # Append the predicted value to the end of the data
    X_scaled = np.append(X_scaled, predicted_value_scaled, axis=0)

    # Update last_data_point for the next iteration
    last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)

print("Predicted Close Prices for Next 7 Days:", predicted_values)


1/1 [==============================] - 0s 25ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
# Convert predicted values to original scale
predicted_values_original = scaler_y.inverse_transform(predictions_scaled).flatten()

# Convert actual values to original scale
actual_values_original = scaler_y.inverse_transform(y_test).flatten()

# Calculate the absolute percentage error
absolute_percentage_error = (
    np.abs(predicted_values_original - actual_values_original) / actual_values_original
)

# Define a threshold for considering a prediction as correct
threshold = 0.05  # For example, considering predictions within 5% of the actual value as correct

# Count correct and incorrect predictions
correct_predictions = np.sum(absolute_percentage_error <= threshold)
incorrect_predictions = np.sum(absolute_percentage_error > threshold)

# Calculate accuracy
total_predictions = len(absolute_percentage_error)
accuracy = correct_predictions / total_predictions

In [ ]:
accuracy

0.9994196169471852